## data

In [1]:
import pandas as pd
import os
import openpyxl

In [2]:
os.chdir(r'E:/STU/Lab/(230105)QFD/Practice(Bosch)/code')
os.getcwd()

'E:\\STU\\Lab\\(230105)QFD\\Practice(Bosch)\\code'

In [3]:
cr = pd.read_csv("amazon_WAN282X1GB.csv")

,Unnamed: 0,Dates,Ratings,Titles,Bodys
0,0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...
1,1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...
2,2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...
3,3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...
4,4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...
...,...,...,...,...,...
288,288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...
289,289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...
290,290,2021-02-16,5,Good washer,Does the job A+++
291,291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver..."


In [5]:
cr.drop(['Unnamed: 0'],axis=1,inplace=True)

In [6]:
cr.columns = ['Date','Star','title','body']
cr

,Date,Star,title,body
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...
...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...
290,2021-02-16,5,Good washer,Does the job A+++
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver..."


## translate
- 한국어 불용어 처리가 잘 안됨
- 영어로 바꿔보면 어떨까
- 영어사전은 잘 되어 있는 것 같음

- 생각보다 번역이 잘됨
ex) 아쥬 좋아요! -> i like it (오타 같은건 아예 빼버리는 듯)

In [7]:
import googletrans

def translate(df):
    translator = googletrans.Translator()
    
    for i in range(len(df)):    
        try:
            result = translator.translate(df['body'][i],dest='en')
            print(i,result.text)
            df['body'][i] = result.text
        except:
            continue

In [8]:
translate(cr)

In [9]:
cr

,Date,Star,title,body
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...
...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...
290,2021-02-16,5,Good washer,Does the job A+++
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver..."


In [10]:
cr.to_csv("cr_translate_WAN282X1GB.csv")

## 전처리?

In [11]:
import pickle
import re
import nltk
from nltk.corpus import stopwords
from tqdm import tqdm

cr['clean_text'] = pd.Series(dtype='object')
cr['stopwords_after'] = pd.Series(dtype='object')
cr['pos_tag'] = pd.Series(dtype='object')
cr['NN'] = pd.Series(dtype='object')
cr['lemmatization'] = pd.Series(dtype='object')

stops = stopwords.words('english')
stops.append('machine')
stops.append('product')
stops.append('bosch')

def clean_text(texts): 
    corpus = []
    
    for i in tqdm(range(0, len(texts))):
        
        body = texts[i]
        
        body = re.sub('[^a-zA-Z]', ' ', body) # 특수문자 제거 
        body = body.lower().split() # 대문자를 소문자로 변경, 문장을 단어 단위로 구분
        
        cr['clean_text'][i] = body
        
        no_stops = [word for word in body if not word in stops] # 불용어 제거
        cr['stopwords_after'][i] = no_stops
        
        tokens_pos = nltk.pos_tag(cr['stopwords_after'][i]) # pos tagging (품사 태깅)
        cr['pos_tag'][i] = tokens_pos
        
        NN_words = [] # 명사는 NN을 포함하고 있음을 알 수 있음(명사만 추출)
        for word, pos in tokens_pos:
            if 'NN' in pos:
                NN_words.append(word)
                cr['NN'][i] = NN_words
                
        wlem = nltk.WordNetLemmatizer() # Lemmatization(원형(lemma) 찾기) # nltk에서 제공되는 WordNetLemmatizer을 이용
        lemmatized_words = []
        for word in NN_words:
            new_word = wlem.lemmatize(word)
            lemmatized_words.append(new_word)
            cr['lemmatization'][i] = lemmatized_words
        
        corpus.append(no_stops) 
        
    return corpus

In [12]:
cr.to_csv("cr_beforeclean_WAN282X1GB.csv")

In [13]:
clean_text(cr['body'])

  0%|                                                                                          | 0/293 [00:00<?, ?it/s]C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['clean_text'][i] = body
C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr['stopwords_after'][i] = no_stops
C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\2666240096.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pyd

[['wife',
  'sight',
  'loss',
  'needed',
  'reliable',
  'easy',
  'use',
  'fits',
  'bill',
  'perfectly'],
 ['old',
  'washer',
  'retire',
  'say',
  'stopped',
  'working',
  'tuesday',
  'yrs',
  'faithful',
  'service',
  'singing',
  'dancing',
  'new',
  'washer',
  'arrived',
  'today',
  'friday',
  'looks',
  'good',
  'easy',
  'use',
  'though',
  'used',
  'couple',
  'times',
  'told',
  'even',
  'night',
  'time',
  'setting',
  'well',
  'worth',
  'money',
  'pounds',
  'install',
  'take',
  'away',
  'old',
  'one',
  'packaging',
  'well',
  'impressed',
  'service',
  'company',
  'thankyou',
  'speed',
  'delivery',
  'delivery',
  'people',
  'courteous',
  'considerate',
  'x'],
 ['expensive',
  'washing',
  'owned',
  'honestly',
  'say',
  'first',
  'one',
  'looks',
  'like',
  'clothes',
  'came',
  'really',
  'clean',
  'even',
  'short',
  'cycle',
  'favourite',
  'modes',
  'short',
  'ones',
  'like',
  'fact',
  'stop',
  'add',
  'forgotten',
 

In [29]:
cr

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


## LDA

In [30]:
cr.to_csv("cr_cleaning_WAN282X1GB.csv")
cr

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


In [31]:
cr

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


In [32]:
cr.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 293 entries, 0 to 292
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             293 non-null    object
 1   Star             293 non-null    int64 
 2   title            293 non-null    object
 3   body             293 non-null    object
 4   clean_text       293 non-null    object
 5   stopwords_after  293 non-null    object
 6   pos_tag          293 non-null    object
 7   NN               274 non-null    object
 8   lemmatization    274 non-null    object
dtypes: int64(1), object(8)
memory usage: 20.7+ KB


In [33]:
cr2 = cr
cr2

,Date,Star,title,body,clean_text,stopwords_after,pos_tag,NN,lemmatization
0,2023-03-12,5,Highly Recommended,My wife has sight loss. We needed a machine th...,"[my, wife, has, sight, loss, we, needed, a, ma...","[wife, sight, loss, needed, reliable, easy, us...","[(wife, NN), (sight, VBD), (loss, NN), (needed...","[wife, loss, use, fits, bill]","[wife, loss, use, fit, bill]"
1,2023-01-20,5,reasonably priced,My old washer I had to retire or should I say ...,"[my, old, washer, i, had, to, retire, or, shou...","[old, washer, retire, say, stopped, working, t...","[(old, JJ), (washer, NN), (retire, NN), (say, ...","[washer, retire, service, washer, today, use, ...","[washer, retire, service, washer, today, use, ..."
2,2023-01-22,4,Would recommend buying this.,This is the most expensive washing machine I’v...,"[this, is, the, most, expensive, washing, mach...","[expensive, washing, owned, honestly, say, fir...","[(expensive, JJ), (washing, NN), (owned, VBN),...","[washing, clothes, cycle, modes, ones, fact, a...","[washing, clothes, cycle, mode, one, fact, add..."
3,2023-02-19,5,Quiet and get the job done well,The series 4 was valuable for small budget. Ov...,"[the, series, was, valuable, for, small, budge...","[series, valuable, small, budget, overall, per...","[(series, NN), (valuable, JJ), (small, JJ), (b...","[series, budget, performance, excellent, featu...","[series, budget, performance, excellent, featu..."
4,2023-01-27,4,Remove Transit Bolts,I read a few of the low rating reviews.They me...,"[i, read, a, few, of, the, low, rating, review...","[read, low, rating, reviews, mention, jumps, a...","[(read, VB), (low, JJ), (rating, NN), (reviews...","[rating, reviews, mention, jumps, cycle, presu...","[rating, review, mention, jump, cycle, presume..."
...,...,...,...,...,...,...,...,...,...
288,2021-02-28,5,So quiet!,Upgraded from my 5 year old Bosch and glad I d...,"[upgraded, from, my, year, old, bosch, and, gl...","[upgraded, year, old, glad, awesome, washing, ...","[(upgraded, JJ), (year, NN), (old, JJ), (glad,...","[year, washing, mode, clothes]","[year, washing, mode, clothes]"
289,2021-06-26,5,"So, so quiet",I had to look to make sure the washing machine...,"[i, had, to, look, to, make, sure, the, washin...","[look, make, sure, washing, working, quiet, cl...","[(look, NNS), (make, VBP), (sure, JJ), (washin...","[look, clothes, time, delivery, persons, job, ...","[look, clothes, time, delivery, person, job, r..."
290,2021-02-16,5,Good washer,Does the job A+++,"[does, the, job, a]",[job],"[(job, NN)]",[job],[job]
291,2021-10-13,4,IT LOOKS GREAT BUT NO INSTALLATION PROVIDED,"It seems to be as advertised, but what was ver...","[it, seems, to, be, as, advertised, but, what,...","[seems, advertised, disappointing, installatio...","[(seems, VBZ), (advertised, JJ), (disappointin...","[installation, option, pay, call, pay, money, ...","[installation, option, pay, call, pay, money, ..."


In [34]:
##길이가 2 이하인 문자 제거
cr2['lemmatization'] = cr2['lemmatization'].replace("[^0-9-_.]","")
cr2 = cr2.dropna()
cr2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274 entries, 0 to 292
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             274 non-null    object
 1   Star             274 non-null    int64 
 2   title            274 non-null    object
 3   body             274 non-null    object
 4   clean_text       274 non-null    object
 5   stopwords_after  274 non-null    object
 6   pos_tag          274 non-null    object
 7   NN               274 non-null    object
 8   lemmatization    274 non-null    object
dtypes: int64(1), object(8)
memory usage: 21.4+ KB


In [37]:
import warnings
from gensim.corpora.dictionary import Dictionary
from gensim import corpora

In [38]:
dictionary = corpora.Dictionary(cr2['lemmatization'])
corpus = [dictionary.doc2bow(text) for text in cr2['lemmatization']]

In [39]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

(0, '0.041*"time" + 0.040*"wash" + 0.025*"program" + 0.023*"spin" + 0.023*"use" + 0.016*"feature" + 0.015*"drum" + 0.014*"door"')
(1, '0.029*"value" + 0.029*"money" + 0.024*"use" + 0.021*"year" + 0.020*"programme" + 0.019*"delivery" + 0.017*"love" + 0.015*"cycle"')
(2, '0.018*"time" + 0.015*"spin" + 0.015*"cycle" + 0.015*"program" + 0.015*"week" + 0.012*"drum" + 0.012*"foot" + 0.010*"use"')
(3, '0.025*"time" + 0.021*"excellent" + 0.020*"delivery" + 0.017*"water" + 0.017*"panel" + 0.013*"problem" + 0.013*"control" + 0.013*"team"')
(4, '0.086*"use" + 0.035*"install" + 0.030*"job" + 0.019*"washer" + 0.018*"day" + 0.016*"amazon" + 0.012*"option" + 0.010*"date"')
(5, '0.043*"quality" + 0.031*"year" + 0.029*"clothes" + 0.022*"wash" + 0.022*"washing" + 0.018*"cycle" + 0.017*"program" + 0.013*"installation"')
(6, '0.068*"cycle" + 0.038*"wash" + 0.033*"spin" + 0.031*"clothes" + 0.023*"delivery" + 0.022*"time" + 0.021*"use" + 0.013*"hour"')


In [40]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


In [41]:
##머신 단어 제거
cr2['lemmatization'] = cr2['lemmatization'].replace("machine","")
cr2 = cr2.dropna()
cr2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 274 entries, 0 to 292
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Date             274 non-null    object
 1   Star             274 non-null    int64 
 2   title            274 non-null    object
 3   body             274 non-null    object
 4   clean_text       274 non-null    object
 5   stopwords_after  274 non-null    object
 6   pos_tag          274 non-null    object
 7   NN               274 non-null    object
 8   lemmatization    274 non-null    object
dtypes: int64(1), object(8)
memory usage: 21.4+ KB


C:\Users\nurip\AppData\Local\Temp\ipykernel_33648\793555062.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cr2['lemmatization'] = cr2['lemmatization'].replace("machine","")


- machine 제거

In [42]:
import gensim

NUM_TOPICS = 7
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

(0, '0.024*"delivery" + 0.021*"installation" + 0.019*"use" + 0.019*"door" + 0.017*"time" + 0.016*"drum" + 0.016*"love" + 0.014*"guy"')
(1, '0.033*"delivery" + 0.027*"use" + 0.026*"wash" + 0.021*"service" + 0.021*"amazon" + 0.019*"program" + 0.017*"cycle" + 0.013*"lot"')
(2, '0.056*"cycle" + 0.029*"wash" + 0.028*"time" + 0.022*"clothes" + 0.019*"year" + 0.015*"programme" + 0.015*"lot" + 0.013*"day"')
(3, '0.047*"wash" + 0.045*"time" + 0.031*"value" + 0.026*"day" + 0.025*"money" + 0.013*"performance" + 0.013*"program" + 0.013*"spin"')
(4, '0.029*"quality" + 0.028*"spin" + 0.026*"year" + 0.025*"clothes" + 0.017*"job" + 0.017*"delivery" + 0.017*"thing" + 0.016*"cycle"')
(5, '0.061*"use" + 0.033*"program" + 0.029*"cycle" + 0.026*"wash" + 0.021*"washer" + 0.021*"programme" + 0.015*"spin" + 0.015*"hour"')
(6, '0.030*"use" + 0.023*"excellent" + 0.019*"clean" + 0.019*"amazon" + 0.014*"plumber" + 0.014*"install" + 0.014*"item" + 0.014*"wife"')


In [43]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(


- machine, product, bosch 제거

In [44]:
import gensim

NUM_TOPICS = 11
ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = NUM_TOPICS, id2word=dictionary, passes=15) #passes는 알고리즘의 동작 횟수
topics = ldamodel.print_topics(num_words=8) #총 8개의 단어만 출력

for topic in topics:
    print(topic)

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

...

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
C:\Users\nurip\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:826: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(

(0, '0.057*"clothes" + 0.047*"cycle" + 0.028*"wash" + 0.028*"delivery" + 0.017*"time" + 0.016*"price" + 0.015*"machine" + 0.014*"hour"')
(1, '0.035*"job" + 0.018*"week" + 0.018*"time" + 0.018*"refund" + 0.017*"clothes" + 0.016*"washer" + 0.014*"use" + 0.012*"lot"')
(2, '0.032*"wash" + 0.024*"day" + 0.024*"floor" + 0.016*"time" + 0.016*"cycle" + 0.016*"use" + 0.016*"purchase" + 0.016*"delivery"')
(3, '0.058*"time" + 0.043*"wash" + 0.033*"spin" + 0.019*"cycle" + 0.017*"program" + 0.015*"energy" + 0.015*"speed" + 0.012*"start"')
(4, '0.057*"money" + 0.053*"value" + 0.033*"use" + 0.033*"day" + 0.022*"time" + 0.021*"wash" + 0.021*"clothes" + 0.017*"cycle"')
(5, '0.027*"use" + 0.023*"delivery" + 0.023*"service" + 0.020*"cycle" + 0.019*"wash" + 0.017*"lot" + 0.017*"time" + 0.017*"result"')
(6, '0.109*"use" + 0.059*"cycle" + 0.030*"program" + 0.029*"load" + 0.022*"love" + 0.021*"wash" + 0.019*"drum" + 0.019*"job"')
(7, '0.037*"spin" + 0.022*"programme" + 0.021*"time" + 0.017*"use" + 0.016*"ite

In [45]:
import pyLDAvis.gensim_models

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldamodel, corpus, dictionary)
pyLDAvis.display(vis)

C:\Users\nurip\Anaconda3\lib\site-packages\pyLDAvis\_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
